# Modèle de Mélange Gaussien et algorithme *Expectation-Maximization*

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * aux modèles à variables cachées.
* Être capable&nbsp;:
    * d'implémenter une GMM avec $\texttt{sklearn}$.



 ----

# I. Introduction

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal 
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('ggplot')

class MultivariateGaussianMixture(object):        
    def __init__(self, n_components=2, d=2, probas=None):
        self.n_components = n_components
        self.sigma = []
        self.mu = []
        
        if probas is None:
            self.probas = np.ones(n_components)/n_components
        else:
            self.probas = np.array(probas)
            s = np.sum(self.probas)
            assert s < 1.0 + 1e-10 and s > 1.0 - 1e-10, "Probabilites should add up to 1, but s = %f"%s

        scale = 1
        scater = 10
        sigma = scale * np.diag([1.0, 0.1])
        for k in range(n_components):
            Q = self.random_rotation(d)
            self.sigma.append(np.dot(np.dot(Q, sigma ), Q.T))
            self.mu.append(np.random.uniform(-scater,scater,size=d))
            
    def random_rotation(self, d):
        Q, _ = np.linalg.qr(np.random.random((d,d)))
        return Q
    
    def sample_categorical(self, N):
        idx_sort = np.argsort(-self.probas)
        probas_sort = self.probas[idx_sort]

        c_probas = np.array([np.sum(probas_sort[:k+1]) for k in range(self.n_components)])   
        X = np.random.uniform(0,1, size=N)
        for i,x in enumerate(X):
            c = c_probas-x
            X[i] = idx_sort[self.n_components-len(c[c>=0])]
        return X.astype(np.uint)

    def generate_samples(self, N):
        Z = self.sample_categorical(N)
        return np.array([np.random.multivariate_normal(self.mu[z], self.sigma[z]) for z in Z]), Z
    
    
def plot(X, c=None, means=None, gmd=None, n_std=3, cumul=True):
    # Plot the dataset
    if c is not None:
        plt.scatter(X[:, 0], X[:, 1], c=c)
    else:
        plt.scatter(X[:, 0], X[:, 1])
        
    # Plot the means
    if means is not None:
        for m in means:
            plt.scatter([m[0]], [m[1]], s=55, color='red')

    # Plot the concentric ellipses to vizualize covariance matrixes of the components
    if gmd is not None:
        stds = [n_std]
    if cumul:
        stds = [i for i in range(1, n_std + 1)]
    
    for std in stds:
        # For each component, we look for the maximum variance direction carried by the eigen vectors
        # The parameters of the elipse are defined by the std deviations ssociated to the sqrt of the eigen values.
        for mu_k, sigma_k in zip(gmd.mu, gmd.sigma):
            e,w = np.linalg.eig(sigma_k)
            top_e = np.argsort(-e)[0]

            a=std*np.sqrt(e[top_e])
            b=std*np.sqrt(e[top_e-1])

            t = np.linspace(-20, 20,100)
            Ell = np.array([a*np.cos(t) , b*np.sin(t)])  

            cos_rot=w[top_e][0]/np.linalg.norm(w[top_e])
            sin_rot=np.sqrt(1-cos_rot**2)
            R_rot = np.array([[cos_rot , -sin_rot],[sin_rot ,cos_rot]])

            Ell_rot = np.zeros((2,Ell.shape[1]))
            for i in range(Ell.shape[1]):
                Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

            plt.plot(mu_k[0]+Ell_rot[0,:] , mu_k[1]+Ell_rot[1,:], color="black", lw=0.1)

MGM = MultivariateGaussianMixture(n_components = 6)
X, Z = MGM.generate_samples(1000)

plot(X, Z, gmd=MGM)
plt.axis('off')
plt.show()

Dans ce chapitre nous allons considérer des méthodes d'apprentissage non-supervisées probabilistes et plus particulièrement des modèles génératifs dont l'idée générale est de considérer l'échantillon de données observées $\mathcal{S}_n = \{x_1,  \dots, x_n\}$ comme des variable aléatoires i.i.d de processus générateur $p_{\mathcal{D}} : x \sim p_{\mathcal{D}}$. Le cas du mélange gaussien est intéressant car il part du principe que nos échantillons sont simulées selon une loi dont la densité a la forme suivante&nbsp;:

$$q(x;\theta)=\sum_{j=1}^K\pi_jf(x;\mu_j,\sigma_j),$$

où $\theta=\{\pi, \mu, \sigma\}$. Le processus générateur d'un tel modèle, comme nous allons le voir, fonctionne de la manière suivante&nbsp;:

1.  On simule, selon une catégorielle de paramètre $\pi$,
2.  On simule selon la loi $f$ associée au "groupe" choisi précédemment.


Notre objectif est de calculer $\theta$ tel que la vraisemblance de nos données $\mathcal{S}_n$ est maximisée. Cela se passe par des algorithmes comme le *Expectation-Maximization* qui introduiront une variable latente $z$ indiquant le "groupe" utilisé afin de générer nos échantillon. Dans ces approches, les représentations des données seront considérées comme ces nouvelles variables aléatoires $z$ qu'on appellera des variables latentes (ou cachées) qui prendront leur valeur conjointement avec les données observées $x$.

**<span style='color:green'> Loi et processus générateur</span>** 
On utilisera par abus de langage l'expression $p$ ou $q_{\theta}$ pour dénoter à la fois le processus générateur des données, la loi de probabilité lui correspondant et la densité de probabilité associée.


 ----
## A. Généralités sur les modèles génératifs
Nous pouvons dans un premier temps introduire un objectif général des modèles non-supervisés génératifs. Il s'agit de trouver dans une famille paramétrique de distributions de probabilité $\mathcal{Q}=\{q_\theta, \theta\in\mathbb{R}^p\}$ une paramétrisation $q_{\theta}(x) = q(x|{\theta})$ qui "explique le mieux" les données observées. Comme pour le cas de l'apprentissage supervisé, nous pouvons considérer le critère de maximum de vraisemblance. Nous cherchons donc&nbsp;:


$$\theta_{\text{ML}} = \arg \max_{\theta} \Big[q(\mathcal{S}_n|\theta)\Big] = \arg \max_{\theta } \Big[\prod_i q(x_i|\theta)\Big].$$


Ou bien, de manière équivalente, le maximum de la log-vraissemblance&nbsp;:

$$\theta_{\text{ML}} = \arg \max_{\theta} \Big[\sum_i \ln\Big(q(x_i|\theta)\Big)\Big].$$

**<span style='color:blue'> Exercice</span>** Soit $\mathcal{S}_n\sim \mathcal{N}(\mu,1)^n$ un échantillon simulé selon une loi normale de variance $1$ et de moyenne $\mu$. Quel est le $\mu$ qui maximise la vraisemblance ?


 ----


## B. Les variables cachées
Le calcul de ce maximum de vraisemblance dans le cas d'un modèle de mélange nécessite d'introduire de nouvelles variables&nbsp;: les variables cachées. Ces dernières seront les représentations de nos observations. Cela se fait en démarginalisant&nbsp;

$$q_{\theta}(x) = \int q_{\theta}(x,z)dz = \int q_{\theta}(z) q_{\theta}(x|z)dz$$


**<span style='color:green'> Remarque</span>** 
On souhaite éviter de tomber sur une distribution où les variables $z$ et $x$ sont indépendantes&nbsp;:

$$q_{\theta}(x,z) = q_{\theta}(x)q_{\theta}(z) \Leftrightarrow q_{\theta}(z|x) = q_{\theta}(z)$$

car on souhaite décomposer notre loi initiale en atomes qui seraient plus faciles à estimer. Pour le mélange, si on "connait" le groupe d'un de nos points (ce serait notre $z$), alors on peut plus facilement estimer la loi du groupe (i.e. c'est l'estimation simple de la loi et non du mélange complet). De plus, dans le cas où les $x$ seraient indépendants des $z$, ce dernier ne pourrait en aucun cas être une bonne représentation.


 ----

Nous allons donc chercher à ajouter une variable latente $z$ pertinente ! Dans le cas de variables cachées discrètes (comme c'est le cas dans ce TP)&nbsp;:


$$q_{\theta}(x) = \sum_{k=1}^K q_{\theta}(z=k)q_{\theta}(x|z=k)$$


Nous allons donc chercher à maximiser la log-vraissemblance de la forme&nbsp;:

$$\theta_{\text{ML}} = \arg\max_{\theta } \Big[\sum_i \ln\Big(\sum_{k=1}^c q_{\theta}(z=k)q_{\theta}(x_i|z=k)\Big)\Big]$$


**<span style='color:green'> Remarque</span>** 
Une façon de voir les variables cachées qui va nous intéresser dans le cadre de l'apprentissage de représentation est comme des facteurs explicatifs des données. C’est-à-dire un nombre restreint de variables telles qu'on peut expliquer/prédire une observation sachant ces variables. On peut voir ça comme des facteurs causaux tels que si on connait la cause première d'une chose on peut en déduire une observation complexes qui en découle ou du moins une part significative. En ce sens, les facteurs explicatifs peuvent être considérés comme une abstraction, une représentation plus structurée et compacte des observations. Ainsi une manière de voir le processus générateur des données est d’échantillonner la cause, la valeur du facteur explicatif $z_i \sim p(z)$, puis sachant la cause, échantillonner l'observation qui en découle par la conditionnelle $x_i \sim p(x |z=z_i)$.


 ----

# II. Le modèle de mélange Gaussien vu sous l'angle des modèles à variables cachées

La famille des mixtures de distributions mettent en jeu des densités de probabilité sous la forme d'une combinaison pondérée de densités d'une même famille. Dans ce TP nous allons nous concentrer sur le cas des mixtures de gaussiennes multi-variées. Les modèles de mixtures de gausiennes consistent à modéliser une densité de probabilité où les données sont distribuées localement selon des distributions normales multi-variées. Dans ce cas, l'indice de la gaussienne à laquelle apaprtient une donnée particulière est la variable cachée qui lui est associée. La totalité de l'information résiduelle est ensuite portée par la densité de probabilité de la composante gaussienne. En notant $z=k$ l'identifiant d'une gausienne qui possède ces paramètres propres $(\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$, la densité de probabilité du processus générateur des données prend la forme suivante&nbsp;:


$$p(\boldsymbol{x}; \boldsymbol{\mu},\boldsymbol{\Sigma}) = \sum_{Z=k}^K p(z=k) f(\boldsymbol{x};\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$


où on notera par la suite $p(z=k) = \pi_k$ qui correspond à la probabilité a priori de la $k$-ième composante (on a donc $\sum_k \pi_k = 1$) et $f(\boldsymbol{x};\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ correspond à la densité de probabilité de la loi normale multi-variée $\mathcal{N}(\boldsymbol{x}|\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ de moyenne et matrice de variance-covariance $(\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$&nbsp;:

$$f(\boldsymbol{x};\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) = \frac{1}{(2\pi)^{\frac{d}{2}}|\boldsymbol{\Sigma}_k|^{\frac{1}{2}}} \exp^{-\frac{1}{2}(\boldsymbol{x} - \boldsymbol{\mu}_k)^T)\boldsymbol{\Sigma}_k^{-1}(\boldsymbol{x} - \boldsymbol{\mu}_k)}$$


Il s'agit de la densité de probabilité d'observer une variable $x$ sachant la valeur de la variable cachée que l'on pourra noter $f_k(\boldsymbol{x})$ par la suite par soucis de lisibilité. On peut aussi calculer quelques quantités d'intérêt qui vont nous servir par la suite comme la densité de probabilité de la variable cachée a posteriori de l'observation $x$ grace à la formule de Bayes&nbsp;:

$$p(z=k|x) = \frac{p(z=k)p(x|z=k)}{\sum_j p(z=j)p(x|z=j)} = \frac{\pi_k f(\boldsymbol{x};\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}{\sum_j \pi_j f(\boldsymbol{x};\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)}$$

## A. Echantillonage et visualisation d'un mélange Gaussien

Comme expliqué précédemment, on peut donc voir le processus générateur correspondant à une mixture gaussienne comme se structurant en deux étapes. La première étape consiste en une variable catégorielle à $K$ valeurs dont les probabilités respectives sont les $\pi_k$. Puis connaissant la valeur tirée $k$; il s'agit ensuite d’échantillonner selon une loi normale multivariée de paramètres $(\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$. Le code ce dessous nous permet selon ce procédé d’échantillonnage de générer et visualiser un jeu de données distribué selon une mixture de gaussienne ou les composantes ont été choisies avec des paramètres aléatoires.

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal 
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('ggplot')

class MultivariateGaussianMixture(object):        
    def __init__(self, n_components=2, d=2, probas=None):
        self.n_components = n_components
        self.sigma = []
        self.mu = []
        
        if probas is None:
            self.probas = np.ones(n_components)/n_components
        else:
            self.probas = np.array(probas)
            s = np.sum(self.probas)
            assert s < 1.0 + 1e-10 and s > 1.0 - 1e-10, "Probabilites should add up to 1, but s = %f"%s

        scale = 1
        scater = 10
        sigma = scale * np.diag([1.0, 0.1])
        for k in range(n_components):
            Q = self.random_rotation(d)
            self.sigma.append(np.dot(np.dot(Q, sigma ), Q.T))
            self.mu.append(np.random.uniform(-scater,scater,size=d))
            
    def random_rotation(self, d):
        Q, _ = np.linalg.qr(np.random.random((d,d)))
        return Q
    
    def sample_categorical(self, N):
        idx_sort = np.argsort(-self.probas)
        probas_sort = self.probas[idx_sort]

        c_probas = np.array([np.sum(probas_sort[:k+1]) for k in range(self.n_components)])   
        X = np.random.uniform(0,1, size=N)
        for i,x in enumerate(X):
            c = c_probas-x
            X[i] = idx_sort[self.n_components-len(c[c>=0])]
        return X.astype(np.uint)

    def generate_samples(self, N):
        Z = self.sample_categorical(N)
        return np.array([np.random.multivariate_normal(self.mu[z], self.sigma[z]) for z in Z]), Z
    
    
def plot(X, c=None, means=None, gmd=None, n_std=3, cumul=True):
    # Plot the dataset
    if c is not None:
        plt.scatter(X[:, 0], X[:, 1], c=c)
    else:
        plt.scatter(X[:, 0], X[:, 1])
        
    # Plot the means
    if means is not None:
        for m in means:
            plt.scatter([m[0]], [m[1]], s=55, color='red')

    # Plot the concentric ellipses to vizualize covariance matrixes of the components
    if gmd is not None:
        stds = [n_std]
        if cumul:
            stds = [i for i in range(1, n_std + 1)]

        for std in stds:
            # For each component, we look for the maximum variance direction carried by the eigen vectors
            # The parameters of the elipse are defined by the std deviations ssociated to the sqrt of the eigen values.
            for mu_k, sigma_k in zip(gmd.mu, gmd.sigma):
                e,w = np.linalg.eig(sigma_k)
                top_e = np.argsort(-e)[0]

                a=std*np.sqrt(e[top_e])
                b=std*np.sqrt(e[top_e-1])

                t = np.linspace(-20, 20,100)
                Ell = np.array([a*np.cos(t) , b*np.sin(t)])  

                cos_rot=w[top_e][0]/np.linalg.norm(w[top_e])
                sin_rot=np.sqrt(1-cos_rot**2)
                R_rot = np.array([[cos_rot , -sin_rot],[sin_rot ,cos_rot]])

                Ell_rot = np.zeros((2,Ell.shape[1]))
                for i in range(Ell.shape[1]):
                    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

                plt.plot(mu_k[0]+Ell_rot[0,:] , mu_k[1]+Ell_rot[1,:], color="black", lw=0.1)

MGM = MultivariateGaussianMixture(n_components = 6)
X, _ = MGM.generate_samples(1000)

plot(X)
plt.show()

Ici nous visualisons une mixture de $K=6$ composantes gaussiennes tirées aléatoirement (avec les $\pi_k$ choisis de manière uniforme: $\pi_k=\frac{1}{K} \forall k$).  Dans la suite de ce TP, nous allons voir comment nous pouvons estimer les paramètres d'un modèle génératif de ce type par un algorithme itératif qui, à chaque itération, va proposer un jeu de paramètres qui maximise de plus en plus la log-vraissemblance de ce modèle&nbsp;: l'algorithme Expectation Maximization (EM).

Une application ce genre de modèles génératifs qui vient directement à l'esprit est le clustering. C'est-à-dire  qu'on veut trouver des groupes de données similaires. La particularité ici est que nous choisissons dans une famille de distributions où les clusters sont décrits par des normales multi-variées. On peut donc voir ce modèle de mixtures gaussien comme une sorte de généralisation de l'algorithme *K-means* (nous reviendrons sur le lien entre ces deux concepts un peu plus loin).

## B. L'algortihme EM pour estimer un modèle de mélange gaussien

Dans cette partie nous allons tenter uniquement à partir du jeu de données généré précédement de retrouver les paramètres $\theta^{\star} = \Big\{ \pi_k^{\star}, \boldsymbol{\mu}_k^{\star}, \boldsymbol{\Sigma}_k^{\star} \Big\}_{k \leq K}$ du vrai processus qui nous a permis de générer ce jeu de données. On va donc se placer dans une famille paramétrique de distributions $\mathcal{Q}$ telle que chaque distribution $q_{\theta} \in \mathcal{Q}$ s'écrit sous la forme&nbsp;:


$$q_{\theta}(\boldsymbol{x}) =  \sum_{k=1}^K \pi_k f(\boldsymbol{x};\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$$


Et on cherchera donc à trouver le jeu de paramètres qui va maximiser la log-vraissemblance suivante&nbsp;:

$$\ln\Big(q_{\theta}(\mathcal{S}_n)\Big) = \sum_i^n \ln \Bigg(\sum_k^K \pi_k f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) \Bigg)$$


### i. Première différence avec l'estimation d'une gaussienne multivariée simple

On rappelle que dans le cas d'une loi gausienne multivariée simple, la log-vraissemblance est&nbsp;:

$$\begin{aligned}
\ln\Big(q_{\theta}(\mathcal{S}_n)\Big) &= \sum_i^n \ln \Big(f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)\Big)= \sum_i^n \ln \Bigg( \frac{1}{(2\pi)^{\frac{d}{2}}|\boldsymbol{\Sigma}|^{\frac{1}{2}}}  \exp^{-\frac{1}{2}(\boldsymbol{x}_i - \boldsymbol{\mu})^T)\boldsymbol{\Sigma}^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu})} \Bigg)\\
& = -\frac{1}{2} \Bigg[\sum_i^n d\ln (2\pi) + ln\Big(|\boldsymbol{\Sigma}|\Big) + (\boldsymbol{x}_i - \boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu})\Bigg]
\end{aligned}$$

Il existe donc une forme close où annuler le gradient de cette expression par rapport à $\boldsymbol{\mu}$ et $\boldsymbol{\Sigma}$ conduit à une unique solution.

**<span style='color:blue'> Exercice</span>** Calculer le gradient de la log-vraissemblance de la loi normale multivariée par rapport à ses paramètres $\boldsymbol{\mu}$ et $\boldsymbol{\Sigma}$ puis trouver l'expression qui annule le gradient. 



 ----
**<span style='color:green'> Indices</span>** 
$\nabla_{\Sigma}log(|\Sigma|) = \Sigma^{-1}$ et $\nabla_{\Sigma}\boldsymbol{a}^T\Sigma^{-1}\boldsymbol{b} = -\Sigma^{-1}\boldsymbol{b}\boldsymbol{a}^T\Sigma^{-1}$.


 ----


Dans la suite nous allons voir que ce qui nous arrange dans le cas la loi normale multivariée simple est que le logarithme de l'expression de la log-vraissemblance "annule" l'exponentielle ce qui rend l'expression de la log-vraisemblance et son gradient tractable. Malheureusement, dans le cas du mélange gaussien, une somme d'exponentielles se trouve dans le logarithme et cela rend les expressions beaucoup plus complexes. C'est là que l'algorithme EM prend toute sa pertinence car il va nous permettre malgré tout de trouver un processus itératif qui augmente la log-vraisemblance a chaque itération.

### ii. Optimisation des $\boldsymbol{\mu}_k$

Calculons l'expression du gradient de la log vraissemblance par rapports aux paramètres $\boldsymbol{\mu}_k$&nbsp;:


$$\nabla_{\boldsymbol{\mu}_k} \ln\Big(q_{\theta}\mathcal{S}_n\Big) = \Bigg[\sum_i^n  \frac{\partial ln\Big( \sum_j \pi_j f_j(\boldsymbol{x}_i) \Big)}{\partial f_k(\boldsymbol{x}_i)}  \nabla_{\boldsymbol{\mu}_k} f_k(\boldsymbol{x}_i) \Bigg] = \boldsymbol{0}$$

où l'on a utilisé la règle de composition des dérivées. Le premier facteur nous donne&nbsp;:

$$\frac{\partial ln\Big( \sum_j \pi_j f_j(\boldsymbol{x}_i) \Big)}{\partial f_k(\boldsymbol{x}_i)} = \frac{\pi_k }{\sum_{j}^K \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)}$$


---

Concernant $\nabla_{\boldsymbol{\mu}_k} f_k(\boldsymbol{x}_i)$, introduisons pour des raisons de lisibilité les notations suivnates&nbsp;:

$$f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)  = f_k(\boldsymbol{x}_i) = \frac{e^{g(\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)}}{C(\boldsymbol{\Sigma_k})}$$


où&nbsp;:

$$C(\boldsymbol{\Sigma_k}) = (2\pi)^{\frac{d}{2}}|\boldsymbol{\Sigma}_k|^{\frac{1}{2}}$$

et&nbsp;:

$$g(\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) = g_k(\boldsymbol{x}_i) = -\frac{1}{2}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)^T)\boldsymbol{\Sigma}_k^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k).$$


---

En utilisant à nouveau la règle de dérivation d'une composition, nous obtenons&nbsp;:

$$\begin{aligned}
\nabla_{\boldsymbol{\mu}_k} f_k(\boldsymbol{x}_i) &= \underbrace{\partial_{g_k(\boldsymbol{x}_i)}\Bigg(\frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})}\Bigg)}_{ = \frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})} = f(\boldsymbol{x}_i; \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)} \nabla_{{\boldsymbol{\mu}_k}} g(\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)\\
& = f(\boldsymbol{x}_i; \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)
\underbrace{
    \nabla_{\boldsymbol{\mu}_k}\Bigg[ -\frac{1}{2}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)^T)\boldsymbol{\Sigma}_k^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)\Bigg]
}_{
    =(-\frac{1}{2})(-2)\boldsymbol{\Sigma}_k^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)
}
\end{aligned}$$

En injectant nos résultats intermédiaires, nous obtenons finalement&nbsp;:

$$\begin{aligned}
\nabla_{\boldsymbol{\mu}_k} \ln\Big(q_{\theta}\mathcal{S}^n\Big) &= \Bigg[\sum_i^n  \underbrace{\frac{\pi_k }{\sum_{j}^K \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)}f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) }_{q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)} (-\frac{1}{2}) (-2) \boldsymbol{\Sigma}^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)\Bigg]\\
& = \Bigg[\sum_i^n  q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \boldsymbol{\Sigma}^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)\Bigg] = \boldsymbol{0}
\end{aligned}$$


Remarquons qu'apparait la probabilité a posteriori "que le facteur explicatif $k$ ait causé" l'observation $x_i$.

Nous noterons ce terme $ \gamma_k^i = q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)$. En multipliant par $\boldsymbol{\Sigma}_k$ des deux coté nous pouvons résoudre et nous trouvons&nbsp;:


$$\boldsymbol{\mu}_k = \frac{1}{\sum_i^n \gamma_k^i}\sum_i^n  \gamma_k^i\boldsymbol{x}_i$$

Remarquons qu'il s'agit quasiment du paramètre de maximum de vraisemblance pour une unique loi normale auquel on rajoute la pondération d'appartenance à la $k$-ième composante.

**<span style='color:green'> Lien avec KMeans</span>** 
Constatons un premier lien avec *K-Means* en analysant ce à quoi correspond la probabilité *a posteriori* $\gamma_k^i = q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)$&nbsp;:

$$\gamma_k^i = \frac{\pi_k f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \lambda\boldsymbol{I}) }{\sum_j \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \lambda\boldsymbol{I}) } \propto \frac{\pi_k \exp(-\frac{1}{2\lambda}\lVert\boldsymbol{x}_i - \boldsymbol{\mu}_k\rVert_2^2 )}{\sum_j \pi_j \exp(-\frac{1}{2\lambda}\lVert\boldsymbol{x}_i - \boldsymbol{\mu}_j\rVert_2^2)}$$

où la matrice de covariance est fixée à la matrice identité. Nous pouvons remarquer qu'il s'agit d'un score de la distance au centre de la composante normalisée. Plus une observation se rapproche du centre $\boldsymbol{\mu}_k$ d'une gaussienne, plus la contribution qui domine au dénominateur correspond à la contribution de la $k$-ième composante et donc plus probabilité à posteriori $q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)$ tend vers $1$ et toutes les autres $q_{\theta}(z=j|\boldsymbol{x}_i, \boldsymbol{\mu}_j,\boldsymbol{\Sigma}_j) \forall j \neq k$ tendent vers $0$. Il est possible de voir $\gamma_k^i$ comme un score d'assignation d'une observation $x_i$ à la $k$-ième gaussienne. 

De plus, remarquons que si la variance des composantes tend vers $0$, alors la probabilité *a posteriori* tend vers $1$ pour une observation $\boldsymbol{x}_i$&nbsp;:

$$\lim\limits_{\lambda \rightarrow 0} q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k, \lambda \boldsymbol{I}) = \lim\limits_{\lambda \rightarrow 0} \frac{\pi_k \exp(-\frac{1}{2\lambda}\lVert\boldsymbol{x}_i - \boldsymbol{\mu}_k\rVert_2^2 )}{\sum_j \pi_j \exp(-\frac{1}{2\lambda}\lVert\boldsymbol{x}_i - \boldsymbol{\mu}_j\rVert_2^2)} = r_k^i =
\begin{cases}
1 \text{ si } k = \arg \min_j\lVert\boldsymbol{x}_i - \boldsymbol{\mu}_j\rVert_2^2 \\
0 \text{ sinon }
\end{cases}$$

et nous retrouvons exactement la fonction d'assignation de KMeans et le calcul de $\boldsymbol{\mu}_k$ devient:

$$\boldsymbol{\mu}_k = \frac{1}{\sum_i^n r_k^i}\sum_i^n  r_k^i\boldsymbol{x}_i$$

qui correspond exactement à la procédure de mise à jour des centroïde de l'algorithme de K-Means en calculant le barycentre des points assignés du $k$_ième cluster.

Une autre quantité intéréssante à considérer est la suivante:

$$N_k  = \sum_i^n q_{\theta}(z=k|\boldsymbol{x}_i, \boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) = \sum_i^n \gamma_k^i$$

qui n'est d'autre que l'estimateur de l'espérance du nombre d'observations assignées à la $k$-ième composante gaussienne. Observons l'analogie avec KMeans où $N_k = \sum_i^n r_k^i$ est exactement le nombre de points associés au $k$_ième cluster.


 ----

### iii. Optimisation des $\boldsymbol{\Sigma}_k$
Procédons de manière très similaire pour le calcul des matrices de variance-covariances. La seule différence sera que cette fois-ci, il faudra calculer le gradient de $f_k(\boldsymbol{x}_i)$ par rapport à $\boldsymbol{\Sigma}_k$&nbsp;:

$$\begin{aligned}
\nabla_{\boldsymbol{\Sigma}_k} \ln\Big(q_{\theta}\mathcal{S}_n\Big) &= \Bigg[\sum_i^n  \frac{\partial ln\Big( \sum_j \pi_j f_j(\boldsymbol{x}_i) \Big)}{\partial f_k(\boldsymbol{x}_i)}  \nabla_{\boldsymbol{\Sigma}_k} f_k(\boldsymbol{x}_i) \Bigg] \\
&=  \Bigg[\sum_i^n  \frac{\pi_k  }{\sum_{j}^K \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)} \nabla_{{\boldsymbol{\Sigma}_k}} \Bigg(\frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})}\Bigg) \Bigg]
\end{aligned}$$

En rappellant l'expression de la dérivée d'un quotient $\Big(\frac{u}{v}\Big)' = \frac{u'v-uv'}{v^2}$&nbsp;:


$$\nabla_{{\boldsymbol{\Sigma}_k}} \Bigg(\frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})}\Bigg) = \frac{\nabla_{\boldsymbol{\Sigma}_k} \Big(e^{g_k(\boldsymbol{x}_i)}\Big) C(\boldsymbol{\Sigma_k}) -  e^{g_k(\boldsymbol{x}_i)} \nabla_{\boldsymbol{\Sigma}_k} \Big(C(\boldsymbol{\Sigma_k})\Big)}{C(\boldsymbol{\Sigma_k})^2}$$

avec&nbsp;:

$$\nabla_{\boldsymbol{\Sigma}_k} \Big(e^{g_k(\boldsymbol{x}_i)}\Big) = \underbrace{\partial_{g_k(\boldsymbol{x}_i)}\Big(e^{g_k(\boldsymbol{x}_i)}\Big)}_{e^{g_k(\boldsymbol{x}_i)}} \nabla_{\boldsymbol{\Sigma}_k} \Big(g_k(\boldsymbol{x}_i)\Big)$$

En factorisant par $\frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})} = f_k(\boldsymbol{x}_i)$, et en se rappellant que $\frac{u'}{u} = ln(u)'$, nous obtenons&nbsp;:

$$\begin{aligned}
\nabla_{{\boldsymbol{\Sigma}_k}} \Bigg(\frac{e^{g_k(\boldsymbol{x}_i)}}{C(\boldsymbol{\Sigma_k})}\Bigg)&= f_k(\boldsymbol{x}_i)
\Bigg[
\nabla_{\boldsymbol{\Sigma}_k} \Big(g_k(\boldsymbol{x}_i)\Big) - \underbrace{\frac{\nabla_{\boldsymbol{\Sigma}_k} \Big(C(\boldsymbol{\Sigma_k})\Big)}{C(\boldsymbol{\Sigma_k})}}_{ = \nabla_{\boldsymbol{\Sigma}_k} ln \Big(C(\boldsymbol{\Sigma_k})\Big)}
\Bigg]\\
&= f_k(\boldsymbol{x}_i)
\Bigg[
\nabla_{\boldsymbol{\Sigma}_k} \Big[-\frac{1}{2}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)^T)\boldsymbol{\Sigma}_k^{-1}(\boldsymbol{x}_i - \boldsymbol{\mu}_k)\Big] - \nabla_{\boldsymbol{\Sigma}_k} \Big[ \frac{d}{2} ln(2\pi) + \frac{1}{2}ln\Big(|\boldsymbol{\Sigma}_k| \Big) \Big]
\Bigg]\\
&= f_k(\boldsymbol{x}_i) (-\frac{1}{2})\Bigg[-\boldsymbol{\Sigma}^{-1} (\boldsymbol{x}_i - \boldsymbol{\mu})(\boldsymbol{x}_i - \boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1} + \boldsymbol{\Sigma}_k^{-1}\Bigg]
\end{aligned}$$

Ainsi, afin d'annuler le gradient, nous obtenons&nbsp;:

$$\nabla_{\boldsymbol{\Sigma}_k} \ln\Big(q_{\theta}\mathcal{S}^n\Big)  = -\frac{1}{2} \sum_i^n  \Bigg[ \underbrace{\Bigg[ \frac{\pi_k  f_k(\boldsymbol{x}_i)}{\sum_{j}^K \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)}\Bigg]}_{\gamma_k^i} \Bigg[-\boldsymbol{\Sigma}^{-1} (\boldsymbol{x}_i - \boldsymbol{\mu})(\boldsymbol{x}_i - \boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1} + \boldsymbol{\Sigma}_k^{-1}\Bigg]\Bigg] = \boldsymbol{0}$$


Multiplions à droite et à gauche par $\boldsymbol{\Sigma}_k$ et résolvons&nbsp;:


$$\boldsymbol{\Sigma}_k = \frac{1}{N_k}\sum_i^n  \gamma_k^i (\boldsymbol{x}_i - \boldsymbol{\mu}_k)(\boldsymbol{x}_i - \boldsymbol{\mu}_k)^T$$


Où nous retrouvons, de manière analogue au calculs des $\mu_k$, les expression des matrices de covariances empiriques et où la contribution de chaque échantillon est pondérées par sa probabilité a posteriori d'avoir été généré par la $k$-ième composante gaussienne.

### iv. Optimisation des $\pi_k$
Concernant l'opimisation des $\pi_k$, il ne suffit pas simplement d'annuler le gradient de log vraissemblance car nous avons la contrainte que leur somme sur $k$ doit valoir $1$ (il s'agit d'une distribution de probabilité). Pour cela, nous allons donc considerer un problème d'optimisation sous contrainte. Définissons le Lagrangien $\mathcal{L}\Big( \ln(q_{\theta}\mathcal{S}_n) , \lambda\Big)$ (voir la séquence "L'optimisation") qui intègre cette contrainte&nbsp;:

$$\mathcal{L}\Big( \ln(q_{\theta}\mathcal{S}_n) , \lambda\Big) = \sum_i^n ln \Big(\sum_k \pi_k f_k(\boldsymbol{x}_i) \Big) + \lambda \Big(\sum_k \pi_k - 1\Big)$$


Annuler la dérivée du Lagrangien par rapport à notre multiplicateur de Lagrange $\lambda$ nous permet bien de retrouver notre contrainte&nbsp;:


$$\frac{\partial \mathcal{L}\Big( \ln(q_{\theta}\mathcal{S}_n) , \lambda\Big)}{\partial \lambda} =  \Big(\sum_k \pi_k - 1\Big) = 0 \Leftrightarrow \sum_k \pi_k  =  1$$


Annulons maintenant la dérivée par rapport à un $\pi_k$ donnée&nbsp;:

$$\frac{\partial \mathcal{L}\Big( \ln(q_{\theta}\mathcal{S}_n) , \lambda\Big)}{\partial \pi_k} = \sum_i^n \frac{\partial ln\Big( \sum_j \pi_j f_j(\boldsymbol{x}_i) \Big)}{\partial \pi_k}  + \lambda = \sum_i^n \frac{f_k(\boldsymbol{x}_i)}{\sum_j \pi_j f_j(\boldsymbol{x}_i)} + \lambda = 0$$


En mutipliant par $\pi_k$ des deux cotés et en sommant sur $k$, nous obtenons&nbsp;:

$$\sum_k \pi_k\sum_i^n \frac{f_k(\boldsymbol{x}_i)}{\sum_j \pi_j f_j(\boldsymbol{x}_i)} + \lambda  \sum_k \pi_k= 0 = \sum_i^n \underbrace{\frac{\sum_k \pi_kf_k(\boldsymbol{x}_i)}{\sum_j \pi_j f_j(\boldsymbol{x}_i)}}_{=1} + \lambda  \underbrace{\sum_k \pi_k}_{=1}$$


et le multiplicateur de Lagrange vaut ainsi $\lambda = -n$. Nous avons également&nbsp;:


$$\underbrace{\sum_i^n \frac{\pi_k f_k(\boldsymbol{x}_i)}{\sum_j \pi_j f_j(\boldsymbol{x}_i)}}_{=\sum_i^n \gamma_k^i=N_k} + \lambda \pi_k = 0 \Leftrightarrow \lambda = -\frac{N_k}{\pi_k} = -n$$

ce qui nous donne&nbsp;:

$$\pi_k  = \frac{N_k}{n}$$

C'est ainsi l'estimateur de l'espérance de la fraction de points assignés à la composante $k$.

### V. L'algortihme EM
Les formules précédentes de nous donnent pas une solution en forme close. En effet, ces dernières sont reliées mutuellement&nbsp;:


$$\gamma_k^i = \frac{\pi_k f(\boldsymbol{x}_i;\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) }{\sum_j \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)}.$$


À chaque fois que nous changerons la valeurs de ces paramètres, les probabilités *a posteriori* changerons aussi ce qui donnera lieu à de nouvelles valeurs des paramètres et ainsi de suite.

Nous pouvons néanmoins définir une procédure itérative en alternant ces deux phases de calculs des $\gamma_k^i$ puis des valeurs optimales des paramêtres en gardant fixe les probabilités *a posteriori*. Cela donne lieu à l'algorithme Expectation Maximization (EM) qui dans le cas de l'estimateur de modèles de mélange Gaussien peut se formuler ainsi&nbsp;:

1) **Etape E (Expectation)**&nbsp;: Calculer les valeurs en utilisant la valeur courante des paramètres $\theta(t) = \Big\{ \pi_k(t), \boldsymbol{\mu}_k(t), \boldsymbol{\Sigma}_k(t) \Big\}_{k \leq K}$ (qu'on initialisera aléatoirement pour la première itération)&nbsp;:

    $$\gamma_k^i (t+1) = \frac{\pi_k(t) f(\boldsymbol{x}_i;\boldsymbol{\mu}_k(t), \boldsymbol{\Sigma}_k(t) }{\sum_j \pi_j f(\boldsymbol{x}_i;\boldsymbol{\mu}_j(t), \boldsymbol{\Sigma}_j(t)}$$


    et&nbsp;:

    $$N_k(t+1) = \sum_i \gamma_k^i(t+1)$$


2) **Etape M (Maximization)**&nbsp;: Calculer les nouvelles valeurs des paramètres en utilisant les valeurs des probabilités *a posteriori* calculées à l'étape précédente:

    $$\pi_k(t+1) =   \frac{N_k(t+1)}{n}$$


    $$\boldsymbol{\mu}_k(t+1) = \frac{1}{N_k(t+1)}\sum_i^n  \gamma_k^i(t+1)\boldsymbol{x}_i $$


    $$\boldsymbol{\Sigma}_k(t+1) = \frac{1}{N_k(t+1)}\sum_i^n  \gamma_k^i(t+1) (\boldsymbol{x}_i - \boldsymbol{\mu}_k(t))(\boldsymbol{x}_i - \boldsymbol{\mu}_k(t))^T$$


Etant donné ce que nous avons vu précédement sur le lien entre l'algorithme EM et KMeans, il est possible d'initialiser les paramètres $\big\{\boldsymbol{\mu}_k(0)\big\}_{k \leq K}$ non pas aléatoirement, mais avec les centroïdes calculés par un algorithme KMeans. C'est souvent ce qui est fait en pratique à la fois pour la qualité et le temps de convergence notamment dans les cas où $d$ est grand et que le nombre de paramètres l'est aussi. Par la suite, nous allons implémenter les éléments de base de l'algorithme EM dans le cas du mélange Gaussien et nous utiliserons les deux modes d'initialisation. 

**<span style='color:blue'> Exercice</span>** Dans le code qui suit, remplissez les méthodes *compute_posteriors*, *compute_priors*, *compute_mu*, *compute_sigma* correspondant respectivement aux calculs des $\gamma_k^i(t)$, $\pi_k(t)$, $\boldsymbol{\mu}_k(t)$, $\boldsymbol{\Sigma}_k(t)$. Complétez aussi le code *evaluate_log_likelyhood* qui vous permettra d'évaluer la log-vraissemblance pour les valeurs courantes des paramètres du modèle&nbsp:

$$\ln\Big(q_{\theta(t)}(\mathcal{S}^n)\Big) = \sum_i^n \ln \Bigg(\sum_k^K \pi_k(t) f(\boldsymbol{x}_i;\boldsymbol{\mu}_k(t), \boldsymbol{\Sigma}_k(t)) \Bigg)$$

On utilisera la méthode *multivariate_normal(mu, sigma).pdf(X)* de *scipy.stats* pour calculer les $f(\boldsymbol{x}_i|\boldsymbol{\mu}_k(t), \boldsymbol{\Sigma}_k(t))$.


 ----

In [ ]:
from sklearn.cluster import KMeans
class GMM(object):
    def __init__(self, n_components=2, d=2, n_iter=5, init="kmeans"):
        self.n_components = n_components
        self.n_iter = n_iter
        self.init = init
        
        self.pi = np.ones(n_components)/n_components
        self.sigma  = [20*np.eye(d) for _ in range(n_components)]
        self.mu     = [np.random.uniform(-1,1,size=(d)) for _ in range(n_components)]
        
    def evaluate_log_likelihood(self, X):
        #### Complete the code here #### or die #######################################
        return ...
        ################################################################################
    
    def compute_posteriors(self, X):
        #Compute numerator of the posterior from Baye's Rule
        #### Complete the code here #### or die #######################################
        ...
            
        return ...
        ################################################################################
    
    def compute_priors(self, p):
        #### Complete the code here #### or die #######################################
        return ...
        ################################################################################
               
    def compute_mu(self, X, p, pi):
        #### Complete the code here #### or die #######################################
        return ...
        ################################################################################
    
    def compute_sigma(self, X, p, pi, mu):
        #### Complete the code here #### or die #######################################
        return ...
        ################################################################################
        
    def predict(self, X):
        return np.argsort(-self.compute_posteriors(X), axis = 1)[:, 0]
    
    def fit_predict(self, X):
        LLH = self.fit(X)
        return self.predict(X), LLH
    
    def fit(self, X):
        LLH_history = []
        # Initialize mu with KMeans
        if self.init == "kmeans":      
            self.mu = KMeans(n_clusters=self.n_components, max_iter=1).fit(X).cluster_centers_
        
        # Peform EM iterations
        for i in range(self.n_iter):
            LLH_history.append(self.evaluate_log_likelihood(X))
            # TODO uncomment print("\rIteration: %d - LogLikelyHood = %f"%(i,LLH_history[-1]), end="")
                
            #E step:
            gamma = self.compute_posteriors(X)

            #M step:
            self.pi  = self.compute_priors(gamma)
            self.mu      = self.compute_mu(X, gamma, self.pi)
            self.sigma   = self.compute_sigma(X, gamma, self.pi, self.mu)
        return LLH_history


In [ ]:
n_iter=50
model = GMM(n_iter=n_iter, n_components=6, init="random")
Z, LLH_random = model.fit_predict(X)

model = GMM(n_iter=n_iter, n_components=6, init="kmeans")
Z, LLH_kmeans = model.fit_predict(X)


# Plot LogLikelyHood
plt.figure()
plt.plot([i for i in range(1, len(LLH_random)+1)], LLH_random, 
         label='LogLikeliHood Random init')
plt.plot([i for i in range(1, len(LLH_kmeans)+1)], LLH_kmeans, 
         label='LogLikeliHood KMeans init')
plt.legend()
plt.show()

# Plot gaussian components
plot(X, Z, means=model.mu, gmd=model)
plt.show()


## C. Justification de l'algorithme EM (approfondissement)

Nous allons maintenant montrer que cette procédure qui peut sembler arbitraire possède en fait une justification théorique qui garantit que la vraissemblance de notre modèle croît bien à chaque itération (sauf si on se trouve dejà dans un maximum local). Au-delà de la croissance, il est également possible de démontrer la convergence de cette méthode. Une première chose à constater est que si nous possédions pour chaque observation $\boldsymbol{x}_i$ la valeur de sa représentation associées $\boldsymbol{z}_i$, alors notre objectif serait de maximiser la log-vraissemblance suivante&nbsp;:


$$\hat{\theta} = \arg \max_{\theta} \sum_i^n \ln \Big(q_{\theta}(\boldsymbol{x}_i,\boldsymbol{z}_i)\Big)$$


L'objectif est de trouver la paramétrisation qui maximise la probabilité d'observer conjointement une observation et sa "vraie" variable cachée. il est possible de démontrer que c'est ce qu'on fait en pratique avec l'algorithme EM. Cependant, n'ayant accès à ces variables cachées, nous utilisons la meilleure information disponible, à savoir sa distribution *a posteriori* à l'instant $t$ : $q_{\theta(t)}(\boldsymbol{z} | \boldsymbol{x})$. Nous effectuons en pratique une optimisation de l'espérance de cette log-vraissemblance sur la distribution a posteriori&nbsp;:


$$\theta(t+1) = \arg \max_{\theta} \Bigg[\mathbb{E}_{z_i\sim q_{\theta(t)}(z_i|x_i)} \Bigg[\sum_i^n ln \Big(q_{\theta}(\boldsymbol{x}_i,\boldsymbol{z}_i)\Big)\Bigg]\Bigg].$$


Notons&nbsp;

$$\mathcal{L}(\theta, \theta(t)) = \mathbb{E}_{z_i\sim q_{\theta(t)}(z_i|x_i)} \Bigg[\sum_i^n ln \Big(q_{\theta}(\boldsymbol{x}_i,\boldsymbol{z}_i)\Big)\Bigg]$$


Comme tous les échantillons sont indépendants et identiquement distribués et que $z$ est une variable discrète, nous pouvons réécrire cette quantité de la manière suivante&nbsp;:

$$\theta(t+1) = \arg \max_{\theta} \Bigg[\sum_i^n \sum_k^K q_{\theta(t)}(z_i=k|\boldsymbol{x}_i) \Big[ \ln (q_{\theta}(z_i=k)) + \ln (q_{\theta}(\boldsymbol{x}_i|z_i=k))\Big]\Bigg]$$

Remarquons que cela revient à pondérer chaque terme de la log-vraissemblance associée à chaque valeur de la variable cachée par la probabilité *a posteriori* de cette variable cachée sachant l'observation associée. 

Ainsi, dans le cadre général de traitement des modèles à variables cachées par l'algorithme EM, ce dernier peut se décrire ainsi&nbsp;:
1) **Expectation**&nbsp;: Utiliser la valeur du paramêtre $\theta(t)$ pour en déduire la distribution *a postériori*&nbsp;:

    $$q_{\theta(t)}(z|\boldsymbol{x})$$


2) **Maximization**&nbsp;: Utiliser cette dernière pour définir l'expression&nbsp;:

    $$\mathcal{L}(\theta, \theta(t)) =  \sum_i^n \sum_k^K q_{\theta(t)}(z_i=k|\boldsymbol{x}_i) \Big[ \ln (q_{\theta}(z_i=k)) + \ln (q_{\theta}(\boldsymbol{x}_i|z_i=k))\Big]$$


    qui dépend du paramêtre général $\theta$ et optimiser $\mathcal{L}(\theta, \theta(t))$ par rapport à ce dernier&nbsp;:


    $$\theta(t) = \arg \max_{\theta} \Bigg[\mathcal{L}(\theta, \theta(t))\Bigg]$$


### i. Retour sur le cas du mélange Gaussien
Si l'on se replace dans le cas du mélange Gaussien, en rappellant qu'on appelle $\gamma_k^i = q_{\theta(t)}(z_i=k|\boldsymbol{x}_i)$ la probabilité *a posteriori* de la $k$-ième composante en gardant les paramètres de l'instant $t$ fixes, l'expression à maximiser devient&nbsp;:

$$\begin{aligned}
\mathcal{L}(\theta, \theta(t)) &= \sum_i^n \sum_k^K \gamma_k^i \Big[ ln (\pi_k) + ln \big(f(\boldsymbol{x}_i ; \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)\big)\Big]\\
& = \sum_i^n \sum_k^K \gamma_k^i \Big[ ln (\pi_k) - \frac{d}{2}ln(2\pi) - \frac{1}{2} ln(|\boldsymbol{\Sigma}_k|)- \frac{1}{2} (\boldsymbol{x}_i - \boldsymbol{\mu}_k)^T \boldsymbol{\Sigma}_k^{-1}  (\boldsymbol{x}_i - \boldsymbol{\mu}_k) \Big]
\end{aligned}$$

Il est possible de vérifier que nous retombons bien sur les paramètres obtenus dans la partie précédente.

**<span style='color:blue'> Exercice</span>** Retrouvez le terme $\boldsymbol{\mu}_k(t+1)$ à partir de cette expression&nbsp;:

$$\nabla_{\mu_k} \Big(\mathcal{L}(\theta, \theta(t))\Big) = \sum_i^n\gamma_k^i(t) \Big[- \frac{1}{2} (-2) \boldsymbol{\Sigma}_k^{-1}  (\boldsymbol{x}_i - \boldsymbol{\mu}_k) \Big] = \boldsymbol{0}$$



 ----

Un avantage avec cette formulation est que le terme intervenant dans le logarithme n'étant plus une somme de densité gaussienne mais directement une densité gaussienne, les calculs de gradient sont beaucoup plus faciles à manipuler. 

### ii. Preuve que chaque itération de EM augmente la log-vraissemblance

**<span style='color:green'> Croissance et convergence</span>** 
Montrer que l'algorithme améliore  la fonction objectif (ici la vraisemblance) n'est pas suffisant pour garantir que notre algorithme est bon. Il faudrait démontrer qu'il converge effectivement vers la quantité souhaitée, mais cela est plus compliqué.


 ----
Même si nous avons donné une vision un peu plus générale de l'algorithme EM et que nous avons donné une intuition de sa justification, nous n'avons pas encore montré rigoureusement pourquoi nous somme garantis que procéder ainsi permet effectivement d’accroître la log-vraissemblance $ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)$ de notre modèle génératif à chaque itération (si nous ne sommes pas déjà dans un maximum local).

Pour cela il convient d'exprimer cette log-vraissemblance de la manière suivante&nbsp;:


$$\ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big) = \ln\Big(q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)\Big) -  \ln\Big(q_{\theta}\Big(\mathcal{Z}^n | \mathcal{S}_n\Big)\Big)$$


Où nous notons $\mathcal{Z}_n$ la variable aléatoire "cachée" correspondant à un échantillon de taille $n$. Une astuce nous permet d'écrire cette expression de la manière suivante&nbsp;:

$$\begin{aligned}
\ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big) 
&= \Bigg[\ln\Big(q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)\Big) - \ln\Big(q'\Big(\mathcal{Z}_n\Big)\Big)\Bigg] -\Bigg[- \ln\Big(q'\Big(\mathcal{Z}_n\Big)\Big) +  \ln\Big(q_{\theta}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)\Big)\Bigg]\\
&=\ln\Bigg( \frac{q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)}{q'\Big(\mathcal{Z}_n\Big)}\Bigg) - \ln\Bigg(\frac{q_{\theta}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)}{q'\Big(\mathcal{Z}_n\Big)}\Bigg)
\end{aligned}$$

Où l'on note $q'\Big(\mathcal{Z}_n\Big)$ comme n'importe quelle distribution sur les variables cachées dans $\mathcal{Q}$. En notant que la marginale de cette dernière sur les échantillon s'intègre à $1$, on en déduit en calculant l'espérance sur $q'\Big(\mathcal{Z}^n\Big)$ des deux cotés&nbsp;:


$$\underbrace{\mathbb{E}_{\mathcal{Z}_n \sim q'(z)^n}\Bigg[\ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big)\Bigg]}_{=  \ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big)} = \underbrace{\mathbb{E}_{\mathcal{Z}_n \sim q'(z)^n} \Bigg[\ln\Bigg( \frac{q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)}{q'\Big(\mathcal{Z}_n\Big)}\Bigg)\Bigg]}_{= L(q', \theta)}  \underbrace{-\mathbb{E}_{\mathcal{Z}_n \sim q'(z)^n}\Bigg[ln\Bigg(\frac{q_{\theta}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)}{q'\Big(\mathcal{Z}_n\Big)}\Bigg)\Bigg]}_{= \mathbb{KL}\big(q'||q\big)}$$


soit&nbsp;:

$$\\ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)= L(q', \theta) + \mathbb{KL}\big(q'||q_{\theta}\big)$$

**<span style='color:green'> Divergence de Kullback Liebler</span>** 
Soit $P$ et $Q$ deux distributions de probabilité. La divergence de Kullback Liebler est donnée par&nbsp;

$$\mathbb{KL}\big(P\lVert Q)=\int p(x)\ln\frac{p(x)}{q(x)}dx.$$

Celle-ci est minimale lorsque $P=Q$ est vaut alors $0$. Notez cependant son manque de symétrie&nbsp;:

$$\mathbb{KL}\big(P\lVert Q)\neq \mathbb{KL}\big(Q\lVert P).$$



 ----
où $\mathbb{KL}\big(q'||q_{\theta}\big)$ correspond à la divergence de Kullback Liebler. Pour tout choix de $q'$ et $\theta$ nous avons nécessairement&nbsp;:

$$L(q', \theta)  = \ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big) - \underbrace{ \mathbb{KL}\big(q'||q_{\theta}\big)}_{\geq 0} \leq \ln\Big(q_{\theta}\Big(\mathcal{S}_n\Big)\Big)$$


Et $L(q', \theta)$ est donc forcément un minorant de la log-vraissemblance pour tout choix de $q'$ et $\theta$. 

Ce qui est intéressant, c'est que si dans une première étape (que l'on constatera être exactement l'étape E de l'algorithme EM) on essaie de trouver la distribution $q'$ qui maximise cette borne inférieure en gardant notre valeur courante de $\theta(t)$ fixe, alors nous obtenons&nbsp;:

$$\begin{aligned}
&\arg \max_{q' \in \mathcal{Q}} \Big[ ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big) - \mathbb{KL}\big(q'||q_{\theta(t)}\big) \Big] \\&=
\arg \min_{q' \in \mathcal{Q}} \Big[ \mathbb{KL}\big(q'\Big(\mathcal{Z}^n\Big)||q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big)\big) \Big] \\&= q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big)
\end{aligned}$$

En fixant $q'\Big(\mathcal{Z}^n\Big) = q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big)$, c'est-à-dire comme la distribution a posteriori (en se fixant un vecteur de paramètres en cours $\theta(t)$), alors ce terme de divergence KL dvient nul et on obtient pour $\theta = \theta(t)$&nbsp;:

$$L\Big(q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}^n\Big), \theta(t)\Big) = \ln\Big(q_{\theta(t)}\Big(\mathcal{S}_n\Big)\Big)$$


Rappellons que pour n'importe quelle autre valeur de $\theta \neq \theta(t)$ nous aurons toujours&nbsp;:

$$L\Big(q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big), \theta\Big)  \leq ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)$$

Autrement dit la log-vraissemblance  $\ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)$ et son minorant $L\Big(q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big), \theta\Big)$ sont tangents en la valeur courante du paramètre $\theta(t)$ et par définition la première est toujours strictement au dessus de la deuxième. Par conséquent, une valeur $\theta= \theta(t+1)$ qui accroît $L\Big(q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big), \theta\Big)$ correspondra aussi à une valeur de paramètre permettant d'accoître $\ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)$. 

Nous allons voir que fixer $q'\Big(\mathcal{Z}_n\Big) = q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)$ correspond à l'étape E de l'algorithme EM, maximiser $L\Big(q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big), \theta\Big)$  par rapport à $\theta$(en gardant fixe $q'$) revient exactement à l'étape M de l'algorithme EM. Pour cela, il convient de vérifier que l'expression de $L\Big(q_{\theta(t)}\Big(\mathcal{Z}^n | \mathcal{S}^n\Big), \theta\Big)$ correspond bien à la quantité à maximiser vue précédemment&nbsp;:


$$\mathcal{L}(\theta, \theta(t)) = \mathbb{E}_{z_i\sim q_{\theta(t)}(z_i|x_i)} \Bigg[\sum_i^n ln \Big(q_{\theta}(\boldsymbol{x}_i,\boldsymbol{z}_i)\Big)\Bigg]$$


On développe donc l'expression de $L\Big(q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big), \theta\Big)$&nbsp;:

$$\begin{aligned}
L\Big(q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big), \theta\Big)&= \mathbb{E}_{\mathcal{Z}_n \sim q_{\theta(t)}(z|x)^n}\Bigg[ ln\Bigg( \frac{q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)}{q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)}\Bigg)\Bigg]\\
&= \mathbb{E}_{\mathcal{Z}_n \sim q_{\theta(t)}(z|x)_n}\Bigg[ ln\Bigg( q_{\theta}\Big(\mathcal{S}_n, \mathcal{Z}_n\Big)\Bigg)\Bigg]- \mathbb{E}_{z_i \sim q_{\theta(t)}(z_i|x_i)}\Bigg[ \ln\Big( q_{\theta(t)}(z_i | x_i)\Big)\Bigg]\\
&= \mathbb{E}_{z_i \sim q_{\theta(t)}(z_i|x_i)}\Bigg[\sum_i^n ln\Big( q_{\theta}(x_i, z_i)\Big)\Bigg] - \text{const}\\
&= \mathcal{L}(\theta, \theta(t)) - \text{const}
\end{aligned}$$

Où nous retrouvons le premier terme comme étant exactement égal à la quantité à maximizer dans le cadre de l'algorithme EM vu précédement, et le deuxième terme ne dépend pas de $\theta$ mais seulement de $\theta(t)$ qu'on a fixé à la première étape où l'on à choisi $q'$. On vient donc de montrer que&nbsp;:

1) **étape E**: en fixant :

    $$q'\Big(\mathcal{Z}_n\Big) = q_{\theta(t)}\Big(\mathcal{Z}_n | \mathcal{S}_n\Big)$$


2) **étape M**: en maximisant la log vraissemblance par rapport à theta en gardant fixe ce $q'$:

    $$\theta(t+1) = \arg \max_{\theta} \Bigg[\mathbb{E}_{z_i \sim q_{\theta(t)}(z_i|x_i)}\Bigg[\sum_i^n \ln\Big( q_{\theta}(x_i, z_i)\Big)\Bigg]\Bigg]$$


alors la quantité qu'on maximise est un minorant qui vaut exactement la log-vraissemblance $\ln\Big(q_{\theta}\Big(\mathcal{S}^n\Big)\Big)$. En itérant sur ces deux étapes on garantit donc de maximiser à chaque fois la log-vraissemblance sauf si à un instant $t$ on se trouve avec une valeur de $\theta(t)$ qui maximise localement cette dernière (le gradient sera nul).